# Generate release notes

This notebook automates creating a draft for the latest release notes using our custom generation script and OpenAI API. All you need to do is have a valid OpenAI API key and the URLs of latest GitHub release tags you want to include.

Release information is extracted from the release tags, and then sorted by label. Then, we put the information through OpenAI for preliminary editing. 

After running the notebook, you can locate the new generated release notes added to the `~/site/releases` folder for you to clean up further, along with a live preview of the site to get you started.

## Contents  
- [Prerequisites](#toc1_)    
- [Setting up](#toc2_)    
  - [Import release generation script](#toc2_1_)    
  - [Set up OpenAI API](#toc2_2_)    
  - [Create categories from labels](#toc2_3_)    
  - [Collect GitHub URLs](#toc2_4_)    
  - [Set the release date](#toc2_5_)    
- [Extracting PR information](#toc3_)    
  - [Create release folder and file](#toc3_1_)    
  - [Add the date to release notes ](#toc3_2_)    
  - [Set up release notes components](#toc3_3_)    
  - [Set the repository and tag name](#toc3_4_)    
  - [Extract PRs from each URL](#toc3_5_)    
  - [Load PR data](#toc3_6_)    
- [Drafting the release notes](#toc4_)    
  - [Edit the release notes body](#toc4_1_)    
  - [Fetch automated GitHub PR summary](#toc4_2_)    
  - [Edit each title](#toc4_3_)    
  - [Set labels for each PR](#toc4_4_)    
  - [Assign PR details to PR](#toc4_5_)    
  - [Assemble release notes](#toc4_6_)    
- [Adding release notes to the docs](#toc5_)    
  - [Write release notes to file](#toc5_1_)    
  - [Update sidebar](#toc5_2_)    
  - [Update index](#toc5_3_)    
- [Next steps](#toc6_)    
  - [Show files to commit](#toc6_1_)    
  - [Preview release notes](#toc6_2_)    
  - [Wrap it up](#toc6_2_)  

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## Prerequisites 

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Refer our release notes guide for the full steps needed before running this notebook:</b></span>
<br></br>
<a href="https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4" style="color: #DE257E;"><b>Create Release Notes</b></a></div>

This process assumes you have knowledge of how to work with GitHub, including managing release tags, sending commits, and creating pull requests. 

- [ ] You should be on a separate branch associated to the Story for the release notes before you begin.
- [ ] You need to have a valid `OPENAI_API_KEY` in your `.env` file. Your `.env` file should preferably live in your `documentation` repo root folder so you don't have to enter in the location of the file when the script runs.
- [ ] You need to have the GitHub release tags created and the URLs ready to be inputted.

<a id='toc2_'></a>

## Setting up 

<a id='toc2_1_'></a>

### Import release generation script 

This cell imports our custom `generate_release_objects.py` script for use with a shortened alias, including any necessary dependencies.

In [1]:
import generate_release_objects as gro

<a id='toc2_2_'></a>

### Set up OpenAI API 

First, you'll be asked to indicate the location of the `.env` file where your OpenAI API key lives so it can return the secrey key. 

**Leave blank to go with the default path:** `../.env`

In [2]:
gro.env_location = gro.get_env_location()
gro.setup_openai_api(gro.env_location)

Using .env file location: ../.env

Detected OpenAI API Key in ../.env


<a id='toc2_3_'></a>

### Create categories from labels 

This cell creates the main sections of the release notes based on the GitHub labels.

`label_hierarchy` shows the order in which updates will be shown. Leave as is to keep the existing order.

**Anything unlabeled will be shoved into `other`, which you will need to review and sort into the right categories when you edit.**

In [3]:
label_hierarchy = ["highlight", "enhancement", "deprecation", "bug", "documentation"]
gro.display_list(label_hierarchy)

Set category hierarchy:

1. highlight
2. enhancement
3. deprecation
4. bug
5. documentation


<a id='toc2_4_'></a>

### Collect GitHub URLs 

Running this cell will prompt you to enter your GitHub release URLs. Keep pasting them in until you're done, then press enter again.

**Example release URL:** https://github.com/validmind/documentation/releases/tag/v2.4.4

In [4]:
gro.github_urls = gro.collect_github_urls() 
gro.count_repos(gro.github_urls)

https://github.com/validmind/frontend/releases/tag/v1.27.17 added

ERROR: Duplicate URL 'https://github.com/validmind/frontend/releases/tag/v1.27.17' not added

https://github.com/validmind/documentation/releases/tag/v2.6.7 added

https://github.com/validmind/documentation/releases/tag/v2.5.25 added

https://github.com/validmind/validmind-library/releases/tag/v2.6.7 added

RELEASE TAGS ADDED BY REPO:

validmind/frontend: 1
validmind/documentation: 2
validmind/validmind-library: 1


<a id='toc2_5_'></a>

### Set the release date 
Running this cell will prompt you to enter the desired release date. 

**The default is 3 business days from today if you leave the prompt empty.**

In [5]:
gro.release_datetime = gro.get_release_date()
gro.formatted_release_date = gro.release_datetime.strftime("%Y-%b-%d").lower()
gro.original_release_date = gro.release_datetime.strftime("%B %-d, %Y")

Release date: 2025-01-01 00:00:00



<a id='toc3_'></a>

## Extracting PR information 

<a id='toc3_1_'></a>

### Create release folder and file

These lines will create a folder inside of `~/site/releases` for the release notes with a new `release-notes.qmd` file. The folder name is the release date, as per our convention. 

**If the directory and file already exists, you will be prompted to confirm whether or not you want to overwrite the contents.**

In [6]:
output_file = gro.create_release_folder(gro.formatted_release_date)

../site/releases/2025-jan-01/release-notes.qmd will be created or overwritten


<a id='toc3_2_'></a>

### Add the date to release notes 
This block writes the specified date as the title of the new release notes file.

In [7]:
gro.create_release_qmd(output_file, gro.original_release_date)

January 1, 2025 added to ../site/releases/2025-jan-01/release-notes.qmd as title


<a id='toc3_3_'></a>

### Set up release notes components 
`release_components` will contain all the components of the release notes in the form of a dictionary. 

Later, we will merge these components together to create the release notes. Again, anything unlabeled will be shoved into `other` at the end of the release notes. 

In [8]:
gro.update_release_components(gro.release_components, gro.categories)

Set up 5 components:


{'highlight': [],
 'enhancement': [],
 'deprecation': [],
 'bug': [],
 'documentation': []}

<a id='toc3_4_'></a>

### Set the repository and tag name 
This block checks every release URL and assigns its repo name, and its associated release tag name.

In [9]:
gro.set_names(gro.github_urls)

Assigning repo and tag names ...

FRONTEND:

URL: https://github.com/validmind/frontend/releases/tag/v1.27.17
 Repo name: validmind/frontend
 Tag name: v1.27.17

DOCUMENTATION:

URL: https://github.com/validmind/documentation/releases/tag/v2.6.7
 Repo name: validmind/documentation
 Tag name: v2.6.7

URL: https://github.com/validmind/documentation/releases/tag/v2.5.25
 Repo name: validmind/documentation
 Tag name: v2.5.25

VALIDMIND LIBRARY:

URL: https://github.com/validmind/validmind-library/releases/tag/v2.6.7
 Repo name: validmind/validmind-library
 Tag name: v2.6.7



<a id='toc3_5_'></a>

### Extract PRs from each URL 
This block gathers all the pull requests from each release URL and stores them within the URL's object data.

In [10]:
gro.extract_urls(gro.github_urls)

Extracting PRs from https://github.com/validmind/frontend/releases/tag/v1.27.17 ...

PR #1085 retrieved.

PR #1086 retrieved.

PR #1087 retrieved.

PR #1088 retrieved.

PR #1089 retrieved.

PR #1090 retrieved.

PR #1091 retrieved.

PR #1083 retrieved.

PR #1093 retrieved.

PR #1094 retrieved.

PR #1079 retrieved.

PR #1097 retrieved.

PR #1098 retrieved.

PR #1103 retrieved.

PR #1104 retrieved.

PR #1105 retrieved.

PR #1080 retrieved.

PR #1099 retrieved.

PR #1095 retrieved.

PR #1108 retrieved.

PR #1110 retrieved.

PR #1109 retrieved.

PR #1111 retrieved.

PR #1112 retrieved.

PR #1113 retrieved.

PR #1114 retrieved.

PR #1118 retrieved.

PR #1100 retrieved.

PR #1101 retrieved.

PR #1081 retrieved.

PR #1076 retrieved.

PR #1107 retrieved.

PR #1106 retrieved.

PR #1092 retrieved.


Extracting PRs from https://github.com/validmind/documentation/releases/tag/v2.6.7 ...

PR #533 retrieved.

PR #540 retrieved.

PR #528 retrieved.

PR #539 retrieved.

PR #538 retrieved.

PR #552 retr

<a id='toc3_6_'></a>

### Load PR data 

This block extracts information from the PR's JSON data, and stores the information in each PR's object data.

In [11]:
gro.populate_data(gro.github_urls)

Storing data from PR #1085 of validmind/frontend ...

Storing data from PR #1086 of validmind/frontend ...

Storing data from PR #1087 of validmind/frontend ...

Storing data from PR #1088 of validmind/frontend ...

Storing data from PR #1089 of validmind/frontend ...

Storing data from PR #1090 of validmind/frontend ...

Storing data from PR #1091 of validmind/frontend ...

Storing data from PR #1083 of validmind/frontend ...

Storing data from PR #1093 of validmind/frontend ...

Storing data from PR #1094 of validmind/frontend ...

Storing data from PR #1079 of validmind/frontend ...

Storing data from PR #1097 of validmind/frontend ...

Storing data from PR #1098 of validmind/frontend ...

Storing data from PR #1103 of validmind/frontend ...

Storing data from PR #1104 of validmind/frontend ...

Storing data from PR #1105 of validmind/frontend ...

Storing data from PR #1080 of validmind/frontend ...

Storing data from PR #1099 of validmind/frontend ...

Storing data from PR #1095 o

<a id='toc4_'></a>

## Drafting the release notes 

<a id='toc4_1_'></a>

### Edit the release notes body 

Using the prompt below, this block feeds the body of each PR to ChatGPT for initial editing, skipping PRs labeled as `internal`. 

**If you find that the output is not quite right, you can edit the prompt and play around with it.**

In [12]:
editing_instructions_body = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

gro.edit_release_notes(gro.github_urls, editing_instructions_body)

Editing content of PR #1079 from validmind/frontend ...

Editing content of PR #1095 from validmind/frontend ...

Editing content of PR #1100 from validmind/frontend ...

Editing content of PR #1076 from validmind/frontend ...

Editing content of PR #1106 from validmind/frontend ...

Editing content of PR #1092 from validmind/frontend ...


Editing content of PR #552 from validmind/documentation ...

Editing content of PR #543 from validmind/documentation ...

Editing content of PR #544 from validmind/documentation ...


Editing content of PR #488 from validmind/documentation ...

Editing content of PR #510 from validmind/documentation ...


Editing content of PR #213 from validmind/validmind-library ...

Editing content of PR #214 from validmind/validmind-library ...

Editing content of PR #249 from validmind/validmind-library ...

Editing content of PR #198 from validmind/validmind-library ...

Editing content of PR #243 from validmind/validmind-library ...

Editing content of PR #21

<a id='toc4_2_'></a>

### Fetch automated GitHub PR summary 

Using the `github-actions` bot, we'll retrieve an auto-generated summary for comparison.

In [13]:
summary_instructions = """ 
    Please turn this PR Summary into a summary for release notes, according to the following guidelines:
    - Use simple and neutral language in the active voice.
    - Change from numbered list format to paragraph-style text.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    """

gro.auto_summary(gro.github_urls, summary_instructions)

Fetching GitHub comment from PR #1079 of validmind/frontend...

Processing PR summary #1079 from validmind/frontend ...

Fetching GitHub comment from PR #1095 of validmind/frontend...

No PR summary found for #1095 from validmind/frontend

Fetching GitHub comment from PR #1100 of validmind/frontend...

Processing PR summary #1100 from validmind/frontend ...

Fetching GitHub comment from PR #1076 of validmind/frontend...

Processing PR summary #1076 from validmind/frontend ...

Fetching GitHub comment from PR #1106 of validmind/frontend...

Processing PR summary #1106 from validmind/frontend ...

Fetching GitHub comment from PR #1092 of validmind/frontend...

Processing PR summary #1092 from validmind/frontend ...


Fetching GitHub comment from PR #552 of validmind/documentation...

Processing PR summary #552 from validmind/documentation ...

Fetching GitHub comment from PR #543 of validmind/documentation...

Processing PR summary #543 from validmind/documentation ...

Fetching GitHub c

<a id='toc4_3_'></a>

### Edit each title 
This block does the same as `Edit the release notes body` for the titles of each PR. The output below will show:
- The original PR title
- The title after some algorithmic changes
- The title after ChatGPT edits it

**If you find that it's not good after editing with ChatGPT, feel free to edit the prompt below.**

In [14]:
editing_instructions_title = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

gro.edit_titles(gro.github_urls, editing_instructions_title)

Editing title for PR #1079 in validmind/frontend...

After trimming: Added the ability to Managed Saved Views

After stripping: Added the ability to Managed Saved Views

After ChatGPT: Added the ability to managed saved views


Editing title for PR #1095 in validmind/frontend...

After trimming: Bump cross-spawn

After stripping: Bump cross-spawn

After ChatGPT: Bump `cross-spawn`


Editing title for PR #1100 in validmind/frontend...

After trimming: Mnt analytics improvements

After stripping: Mnt analytics improvements

After ChatGPT: Mnt analytics improvements


Editing title for PR #1076 in validmind/frontend...

After trimming:  - chore: add UI changes for revision history

After stripping: - chore: add UI changes for revision history

After ChatGPT: Add UI changes for revision history


Editing title for PR #1106 in validmind/frontend...

After trimming: Deleting a finding should remove it from a validation report

After stripping: Deleting a finding should remove it from a valid

<a id='toc4_4_'></a>

### Set labels for each PR 
This block takes the label data from each PR and assigns it to the PR.

In [15]:
gro.set_labels(gro.github_urls)

Attaching labels to PRs ...


PR #1079 from validmind/frontend: ['enhancement']

PR #1095 from validmind/frontend: ['dependencies']

PR #1100 from validmind/frontend: ['enhancement']

PR #1076 from validmind/frontend: ['enhancement']

PR #1106 from validmind/frontend: ['bug']

PR #1092 from validmind/frontend: ['enhancement']


PR #552 from validmind/documentation: ['documentation']

PR #543 from validmind/documentation: ['documentation']

PR #544 from validmind/documentation: ['documentation']


PR #488 from validmind/documentation: ['documentation']

PR #510 from validmind/documentation: ['documentation']


PR #213 from validmind/validmind-library: []

PR #214 from validmind/validmind-library: ['enhancement']

PR #249 from validmind/validmind-library: []

PR #198 from validmind/validmind-library: ['documentation', 'enhancement']

PR #243 from validmind/validmind-library: ['documentation', 'highlight']

PR #213 from validmind/validmind-library: []




<a id='toc4_5_'></a>

### Assign PR details to PR 
This block compiles all the data we found earlier for each PR into one place. 

In [16]:
gro.assign_details(gro.github_urls)

Compiling PR data ...


PR #1079 from validmind/frontend compiled.

PR #1095 from validmind/frontend compiled.

PR #1100 from validmind/frontend compiled.

PR #1076 from validmind/frontend compiled.

PR #1106 from validmind/frontend compiled.

PR #1092 from validmind/frontend compiled.


PR #552 from validmind/documentation compiled.

PR #543 from validmind/documentation compiled.

PR #544 from validmind/documentation compiled.


PR #488 from validmind/documentation compiled.

PR #510 from validmind/documentation compiled.


PR #213 from validmind/validmind-library compiled.

PR #214 from validmind/validmind-library compiled.

PR #249 from validmind/validmind-library compiled.

PR #198 from validmind/validmind-library compiled.

PR #243 from validmind/validmind-library compiled.

PR #213 from validmind/validmind-library compiled.




<a id='toc4_6_'></a>

### Assemble release notes 
Now, we can take all the details we compiled above and append them to our final release notes components. 

Since we want to show features in order of importance, we sort by the priority of the label.

In [17]:
release_components = gro.assemble_release(gro.github_urls, label_hierarchy)

Assembling PR #1079 from validmind/frontend for release notes...

Assembling PR #1095 from validmind/frontend for release notes...

Assembling PR #1100 from validmind/frontend for release notes...

Assembling PR #1076 from validmind/frontend for release notes...

Assembling PR #1106 from validmind/frontend for release notes...

Assembling PR #1092 from validmind/frontend for release notes...


Assembling PR #552 from validmind/documentation for release notes...

Assembling PR #543 from validmind/documentation for release notes...

Assembling PR #544 from validmind/documentation for release notes...


Assembling PR #488 from validmind/documentation for release notes...

Assembling PR #510 from validmind/documentation for release notes...


Assembling PR #213 from validmind/validmind-library for release notes...

Assembling PR #214 from validmind/validmind-library for release notes...

Assembling PR #249 from validmind/validmind-library for release notes...

Assembling PR #198 from valid

<a id='toc5_'></a>

## Adding release notes to the docs 

<a id='toc5_1_'></a>

### Write release notes to file 
Now that `release_components` contains everything we need for the release notes, we can write it to our release notes file.

This step also appends our single-source includes file `/releases/_how-to-upgrade.qmd` automatically to the bottom of the page. 

In [18]:
gro.release_output(output_file, release_components, gro.label_to_category)
gro.upgrade_info(output_file)

Assembled release notes added to ../site/releases/2025-jan-01/release-notes.qmd

Include _how-to-upgrade.qmd added to ../site/releases/2025-jan-01/release-notes.qmd


<a id='toc5_2_'></a>

### Update sidebar 
This block will go into our `_quarto.yml` file and add the new release notes so it shows up on the sidebar of the docsite under the "About" section. 

In [19]:
gro.update_quarto_yaml(gro.release_datetime)

Added new release notes to _quarto.yml, line 173


<a id='toc5_3_'></a>

### Update index 
This block will go into our `index.qmd` file and add the new release notes so it shows up in the "Latest Releases" section, as well as remove the oldest release so that the tiles are uniform. 

In [20]:
gro.update_index_qmd(gro.release_datetime)

Added new release notes to index.qmd, line 49

Removed the oldest release notes entry: '- /releases/2024-aug-13/release-notes.qmd'


<a id='toc6_'></a>

## Next steps 

<a id='toc6_1_'></a>

### Show files to commit 

This block shows you the files changed or added by the notebook you need to commit — it should omit any files in `release-scripts/`. 

In [21]:
%%bash
cd ../
git status | grep -v 'release-scripts/'

On branch beck/demo-release-scripts
Your branch is up to date with 'origin/beck/demo-release-scripts'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   site/_quarto.yml
	modified:   site/index.qmd

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	site/releases/2025-jan-01/

no changes added to commit (use "git add" and/or "git commit -a")


<a id='toc6_2_'></a>

### Preview release notes 
Run this cell to preview the site, including the new release notes you just generated. 

In [ ]:
%%bash
cd ../site
quarto preview

Preparing to preview
[1/4] releases/2025-jan-01/release-notes.qmd
[2/4] training/validator-fundamentals/validator-fundamentals.qmd
[3/4] training/developer-fundamentals/developer-fundamentals.qmd
[4/4] index.qmd

make

Updating Python documentation ...
Copying Python documentation into docs site ...

Watching files for changes
Browse at http://localhost:6978/
GET: /
GET: /releases/2025-jan-01/release-notes.html


<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>When you're done with the preview, please restart the kernel.</b></span>
<br></br>
Make sure that when you commit your changes to remote, do NOT include the changes in the <code>~/release-scripts/</code> folder as that would commit changed notebook with the output cells.</div>

<a id='toc6_3_'></a>

### Wrap it up

**You may want to send a commit up to remote before you begin editing so you have a backup.**

- [ ] At the end of the release notes file, make sure that the notebook appended the `_how-to_upgrade.qmd` include to the very bottom.
- [ ] Make sure that the link to the new release notes displays as expected in the sidebar under `About > Releases`. 
- [ ] Double-check that the `Latest Releases` section on our main documentation landing page includes the new release and looks uniform.
- [ ] Edit the new release notes in accordance with our [internal guide](https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4) to make sure you've completed all the requirements.
- [ ] Make sure any relevant files are committed to remote in preparation for your PR!
